<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/1_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Logger

In [ ]:
def daily_logger():
  local_time = datetime.datetime.now()
  pid = os.getpid()
  with open('../../../../../sever_ip','r') as file:
    server_IP = file.readline().strip()
  #server_IP = socket.gethostbyname(socket.gethostname()) # 獲取當前主機的IP地址
  date = local_time.strftime('%Y-%m-%d')
  if 'log' not in os.listdir('./'):
    os.mkdir('./log')
  logger = logging.getLogger(f'{date}__log') # 创建或获取一个名为 '{date}__log' 的日志记录器（logger）对象，并将其赋值给变量 logger
  if not logger.handlers:
    handler = logging.FileHandler(f'./log/{date}.txt')
    handler.setLevel(logging.INFO)
    formatter = logging.Formater(f'[%(asctime)s__{server_IP}__{pid}]:%(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter) # 将上面创建的格式化器应用到文件处理器(handler)上。 !!!!! 增加這行!!!!!
    logger.addHandler(handler)
  return logger

## train model

In [ ]:
def load_data(data_dir,search='*.png'):
  image_paths = []
  labels = []
  for class_folder_name in os.listdir(data_dir):
    class_folder_path = os.path.join(data_dir, class_folder_name)
    for fpath in glob.glob(os.path.join(class_folder_path, search)):
      image_paths.append(fpath)
      labels.append(class_folder_name)
  return image_paths, labels

In [ ]:
class CustomDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size=16, target_size=(224,224), label_to_index, num_classes, shuffle=True):
        self.image_paths = np.array(image_paths)
        self.labels = np.array(labels)
        self.batch_size = batch_size
        self.target_size = target_size
        #self.augmentation_dict = augmentation_dict
        self.label_to_index = label_to_index
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = []
        batch_labels = []

        for i in batch_indices:
            img_path = self.image_paths[i]
            label = self.labels[i]

            # 加载和预处理图像 (直接在 TensorFlow 中載入和解碼映像。這意味著從一開始圖像就是以 Tensor 的形式存在的)
            img = tf.io.read_file(img_path)
            img = tf.image.decode_image(img, channels=3)
            img = tf.image.resize(img, self.target_size)

            # 应用数据增强
            img = self.apply_augmentation(img, label)/ 255 # 規一化
            batch_images.append(img)
            batch_labels.append(self.label_to_index[label])

        return tf.convert_to_tensor(batch_images), to_categorical(batch_labels, num_classes=self.num_classes)

    def apply_augmentation(self, image):
        # 随机旋转（正负 10 度）
        rotation_degree = random.uniform(-10, 10) #np.random.uniform(-10, 10)
        image = rotate(image, rotation_degree, reshape=False, mode='nearest')
          # reshape=False 保证旋转后的图像大小不变，但这可能导致图像的一部分被裁剪
          # mode 决定了在旋转过程中如何处理图像边界之外的像素

        # 对比度增强
        contrast_factor = 1.5  # 可以根据需要调整这个值
        image = self.adjust_contrast(image, contrast_factor)

        return image

    def adjust_contrast(self, image, contrast_factor):
        """调整图像的对比度"""
        mean = np.mean(image, axis=(0, 1), keepdims=True)
        adjusted = (image - mean) * contrast_factor + mean
        return np.clip(adjusted, 0, 255)


    def on_epoch_end(self):
        self.indices = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indices)



In [ ]:
def retrained_model(data_dir, model_path):
  ogger = daily_logger()
  # 检查 CUDA 是否可用
  cuda_available = tf.test.is_gpu_available(cuda_only=True)
  if cuda_available:
      logger.info("CUDA is available. Using GPU for training.")
  else:
      logger.info("CUDA is not available. Using CPU for training.")
  image_paths, labels = load_data(data_dir)
  test_image_paths, test_labels = load_data(data_dir,reaserch='*/*.png')
  label_to_index = {label: index for index, label in enumerate(sorted(list(set(labels))))}
  train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, stratify=labels ,shuffle=True, random_state=42)

  train_generator = CustomDataGenerator(
    train_paths, train_labels, label_to_index=label_to_index, num_classes=len(label_to_index), shuffle=True)
  val_generator = CustomDataGenerator(
    val_paths, val_labels,   label_to_index=label_to_index, label_to_index=label_to_index, num_classes=len(label_to_index), shuffle=False)
  test_generator = CustomDataGenerator(
    test_paths, test_labels,  label_to_index=label_to_index, label_to_index=label_to_index, num_classes=len(label_to_index), shuffle=False)

  model_ver = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
  model_name = 'Annie_'+model_ver

  # callbacks
  early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
  model_checkpoint = ModelCheckpoint('model_best.h5', save_best_only=True, monitor='val_loss', mode='min') # 可改成.keras格式('model_best.keras')
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001, verbose=1)

  callbacks = [early_stopping, model_checkpoint , reduce_lr]


  model = load_model(model_path) #model_path輸入h5檔
  # 记录训练开始时间
  start_time = time.time()
  history = model.fit(train_generator,
        steps_per_epoch=len(train_paths) // 16,
        validation_data=val_generator,
        validation_steps=len(val_paths) // 16,
        epochs=50,
        callbacks=callbacks)
  # 记录训练结束时间并计算训练总时长
  end_time = time.time()
  training_duration = end_time - start_time

  # 记录训练时间、模型版本和测试准确率
  logger.info(f"Model training completed. Total time: {training_duration:.2f} seconds.")
  logger.info(f"Model version: {model_name}. Test accuracy: {test_accuracy:.4f}")

  test_loss, test_accuracy = model.evaluate(test_generator)

  return model_name, test_accuracy




* 使用類class屬性

In [ ]:
class ModelTrainer:
    def __init__(self):
        self.history = None

    def retrained_model(self, data_dir, model_path):
        # ... 训练过程 ...
        self.history = model.fit(...)
        # ... 其他代码 ...

    def plot_training_history(self):
        if self.history is not None:
            plt.figure(figsize=(12, 5))
            # ... 绘图代码 ...
            plt.show()

# 使用示例
trainer = ModelTrainer()
trainer.retrained_model('data/dir', 'model/path')
trainer.plot_training_history()

In [ ]:
class ImageModelTrainer:
    def __init__(self, data_dir, model_path):
        self.data_dir = data_dir
        self.model_path = model_path
        self.logger = self._daily_logger()

    def load_data(self, search='*.png'):
        # ... 加载数据代码 ...

    def retrained_model(self):
        start_time = time.time()
        self.logger.info("Model retraining started.")

        # ... 加载数据和准备生成器 ...

        model = load_model(self.model_path)
        history = model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=self._get_callbacks())

        training_duration = time.time() - start_time
        self.logger.info(f"Model retraining completed. Duration: {training_duration:.2f} seconds.")
        return history

    def plot_training_history(self, history):
        start_time = time.time()
        self.logger.info("Model retraining started.")

        # 加载数据和准备生成器
        image_paths, labels = self.load_data()
        label_to_index = {label: index for index, label in enumerate(sorted(set(labels)))}
        # 将 label_to_index 字典转换为 DataFrame
        label_to_index_df = pd.DataFrame(list(label_to_index.items()), columns=['Class_Name', 'Index'])
        # 保存为 CSV 文件
        label_to_index_csv_path = 'path/to/save/label_to_index.csv'
        label_to_index_df.to_csv(label_to_index_csv_path, index=False)

        self.logger.info(f"Label to index mapping saved to {label_to_index_csv_path}")



        train_paths, val_paths, train_labels, val_labels = train_test_split(
            image_paths, labels, test_size=0.2, stratify=labels, shuffle=True, random_state=42
        )

        train_generator = CustomDataGenerator(
            train_paths, train_labels, batch_size=16, target_size=(224, 224),
            label_to_index=label_to_index, num_classes=len(label_to_index), shuffle=True
        )
        val_generator = CustomDataGenerator(
            val_paths, val_labels, batch_size=16, target_size=(224, 224),
            label_to_index=label_to_index, num_classes=len(label_to_index), shuffle=False
        )

        # 加载模型
        model = load_model(self.model_path)

        # 获取回调函数
        callbacks = self._get_callbacks()

        # 训练模型
        history = model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=50,
            callbacks=callbacks
        )

        # 获取模型版本
        model_ver = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
        model_name = f"Annie_{model_ver}"

        # 保存模型名到文本文件 (每天生成一個新的)
        model_name_file = f"{model_name}.txt"
        with open(model_name_file, 'w') as file:
            file.write(model_name)

        # 评估测试集
        test_generator = CustomDataGenerator(
            test_paths, test_labels, batch_size=16, target_size=(224, 224),
            label_to_index=label_to_index, num_classes=len(label_to_index), shuffle=False
        )
        test_loss, test_accuracy = model.evaluate(test_generator)
        self.logger.info(f"Test accuracy: {test_accuracy:.4f}")

        training_duration = time.time() - start_time
        self.logger.info(f"Model retraining completed. Duration: {training_duration:.2f} seconds. Model version: {model_ver}")

        return model_ver, test_accuracy.

    def _daily_logger(self):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)

        if not logger.handlers:
            handler = logging.FileHandler(filename=f"./log/{datetime.datetime.now().strftime('%Y-%m-%d')}.log")
            formatter = logging.Formatter('[%(asctime)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
            handler.setFormatter(formatter)
            logger.addHandler(handler)

        return logger

    def _get_callbacks(self):
        model_ver = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        model_name = f"model_{model_ver}.h5"

        early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
        model_checkpoint = ModelCheckpoint(os.path.join(self.model_path, model_name), save_best_only=True, monitor='val_loss', mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001, verbose=1)

        return [early_stopping, model_checkpoint, reduce_lr]

## predict


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import logging
import json

def predict_image_class(model_path, image_path, output_file, target_size=(224, 224)):
    # 获取日志记录器
    logger = daily_logger()

    try:
        # 记录正在加载的模型
        logger.info(f"Loading model from: {model_path}")
        model = load_model(model_path)

        # 加载和预处理图像
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        # 预测图像
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)
        prob_class = {f"class_{i}": prob for i, prob in enumerate(predictions[0])}

        # 将结果写入文件
        with open(output_file, "w") as file:
            file.write(f"Predicted Class: {predicted_class[0]}\n")
            file.write("Probabilities:\n")
            file.write(json.dumps(prob_class, indent=4))

        return predicted_class[0], prob_class

    except Exception as e:
        # 记录错误
        logger.error(f"Error during model prediction: {str(e)}")
        raise

# 使用示例
predict_classname, prob_class = predict_image_class(
    model_path="path/to/your/model.h5",
    image_path="path/to/your/image.jpg",
    output_file="prediction_results.txt"
)

* 然後在這個function中我還要再創一個log檔(使用daily_logger)來記錄，(1)輸入的model_path 還有預測的時間, (2)model如果有出錯的話的error  

In [ ]:
import tensorflow as tf
import numpy as np
import logging
import datetime

def predict_image_class(model_path, image_path, output_txt_path):
    logger = daily_logger()

    try:
        # 载入模型
        model = tf.keras.models.load_model(model_path)
        logger.info(f"Model loaded from {model_path}")

        # 处理图像
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        # 预测
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        class_names = list(model.class_indices.keys())  # 需要模型的类别索引
        predicted_classname = class_names[predicted_class_index]

        # 构建预测概率的字典
        prob_class = {class_names[i]: float(predictions[0, i]) for i in range(len(class_names))}

        # 记录预测时间
        current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        logger.info(f"Prediction made at {current_time}")

        # 将预测结果保存到文本文件
        with open(output_txt_path, 'w') as file:
            file.write(f"Predicted class: {predicted_classname}\n")
            file.write("Probabilities:\n")
            for classname, prob in prob_class.items():
                file.write(f"{classname}: {prob}\n")

        return predicted_classname, prob_class

    except Exception as e:
        logger.error(f"Error during model prediction: {e}")
        raise

# 使用示例
predicted_classname, prob_class = predict_image_class(
    model_path="path/to/your/model.h5",
    image_path="path/to/your/image.jpg",
    output_txt_path="path/to/your/output.txt"
)


* 創一個log檔(使用daily_logger)來記錄，輸入的model_path 還有預測的時間 (class_name有data_frame的csv檔)

In [ ]:
import pandas as pd

def predict_image_class(model_path, image_path, class_names_csv, output_txt_path):
    logger = daily_logger()

    # 记录模型路径和预测时间
    logger.info(f"Predicting image class. Model: {model_path}")
    prediction_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    logger.info(f"Prediction time: {prediction_time}")

    # 从 CSV 文件读取类别名称
    class_names_df = pd.read_csv(class_names_csv)
    class_names = class_names_df.iloc[:, 0].tolist()  # 假设类别名称在第一列

    # 加载模型
    model = load_model(model_path)

    # 加载并预处理图像
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # 预测
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    prob_class = {class_names[i]: float(prob) for i, prob in enumerate(predictions[0])}

    # 获取预测类别名称
    predicted_classname = class_names[predicted_class_index]

    # 记录到文本文件
    with open(output_txt_path, 'w') as file:
        file.write(f"Predicted Class: {predicted_classname}\n")
        file.write(f"Probability of Each Class: {prob_class}\n")

    return predicted_classname, prob_class

# 使用示例
model_path = 'path/to/your/model.h5'
image_path = 'path/to/your/image.jpg'
class_names_csv = 'path/to/your/class_names.csv'
output_txt_path = 'path/to/your/output.txt'
predict_classname, prob_class = predict_image_class(model_path, image_path, class_names_csv, output_txt_path)
